In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.corpus import words
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('words')
from nltk.corpus import stopwords
import re
import string
from gensim.models import Word2Vec
import numpy as np
import sklearn
from node2vec import Node2Vec
from nltk.sentiment import SentimentIntensityAnalyzer
import wordsegment
wordsegment.load()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
graph = nx.read_graphml("drive/MyDrive/web/database_formated_for_NetworkX.graphml")
graph = nx.DiGraph(graph)
graph

In [ ]:
!pip install wordsegment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 14.0 MB/s eta 0:00:00


# Preparation du dataset

In [ ]:
#On sélectionne le contenu des tweets etles infos qui leur sont liées

tweets = {n: graph.nodes[n]['text'] for n in graph.nodes if 'text' in graph.nodes[n]}
priority = {n: graph.nodes[n]['annotation_postPriority'] for n in graph.nodes if 'annotation_postPriority' in graph.nodes[n]}
Retweet = {n: graph.nodes[n]['retweet_count'] for n in graph.nodes if 'retweet_count' in graph.nodes[n]}
Sensitivity = {n: graph.nodes[n]['possibly_sensitive'] for n in graph.nodes if 'possibly_sensitive' in graph.nodes[n]}
Favori = {n: graph.nodes[n]['favorite_count'] for n in graph.nodes if 'favorite_count' in graph.nodes[n]}
Tweet_df = pd.DataFrame({'noeud_tweet' : tweets.keys(), 'Contenu' : tweets.values(), 'Niveau': priority.values(),'Nb_retweet': Retweet.values(),
                        'Sensitivité': Sensitivity.values(), 'Favori' : Favori.values()})
Tweet_df

In [ ]:
#On recupere les infos sur les utilisateurs

Utilisateurs_nom = {n: graph.nodes[n]['name'] for n in graph.nodes if 'name' in graph.nodes[n]}
Nom_secret = {n: graph.nodes[n]['screen_name'] for n in graph.nodes if 'screen_name' in graph.nodes[n]}
Compte_verif = {n: graph.nodes[n]['isVerified'] for n in graph.nodes if 'isVerified' in graph.nodes[n]}
Nb_followers = {n: graph.nodes[n]['followers_count'] for n in graph.nodes if 'followers_count' in graph.nodes[n]}
Nb_favories = {n: graph.nodes[n]['favourites_count'] for n in graph.nodes if 'favourites_count' in graph.nodes[n]}
Nb_emis = {n: graph.nodes[n]['statuses_count'] for n in graph.nodes if 'statuses_count' in graph.nodes[n]}
Nb_public = {n: graph.nodes[n]['listed_count'] for n in graph.nodes if 'listed_count' in graph.nodes[n]}

Df_Utilisateurs1 = pd.DataFrame({'noeud_Util' : Nom_secret.keys(),
                               'Nom perso' : Nom_secret.values(),'Compte_verif' : Compte_verif.values(),
                               'Nb_followers' : Nb_followers.values(), 'Nb_favories' : Nb_favories.values(),
                               'Nb_emis' : Nb_emis.values(), 'Nb_public' : Nb_public.values() })
Df_Utilisateurs2 = pd.DataFrame({'noeud_Util' : Utilisateurs_nom.keys(),
                               'Nom Utilisateur' : Utilisateurs_nom.values()})
Df_Utilisateur = pd.merge(Df_Utilisateurs1,Df_Utilisateurs2, on ='noeud_Util', how = 'inner')
Df_Utilisateur

In [ ]:
#On recupere les categories
postcategory_nodes = {n : graph.nodes[n]["id"] for n in graph.nodes() if graph.nodes[n]['labels'] == ":PostCategory"}
Categori_df = pd.DataFrame({'Evenement' : postcategory_nodes.keys(), 'PostCategory' : postcategory_nodes.values()} )
Categori_df


In [ ]:
#Liaison des utilisateurs au tweet

liasion_utilisateur = [(source,target) for (source,target) in graph.edges() if source in Nom_secret.keys() and target in tweets.keys() ]
liasion_utilisateur_df = pd.DataFrame(liasion_utilisateur)
liasion_utilisateur_df = liasion_utilisateur_df.rename(columns= {0 : 'noeud_Util', 1 : "noeud_tweet"})
liasion_utilisateur_df

In [ ]:
#Liaison des categories au post
liaison = [(source,target) for (source,target) in graph.edges() if target in postcategory_nodes.keys()]
Liason_df = pd.DataFrame(liaison)
Liason_df = Liason_df.rename(columns= {0 : 'noeud_tweet', 1 : "Evenement"})
Liason_df

In [ ]:
#Fusion de tous les dataframe
Df_final = pd.merge(Liason_df, Categori_df, on = "Evenement", how = 'left')
Df_final = pd.merge(Df_final, Tweet_df, on = "noeud_tweet")
Df_info = pd.merge(liasion_utilisateur_df, Df_Utilisateur, on = 'noeud_Util', how = 'inner')
Df_final = pd.merge(Df_info, Df_final, on = 'noeud_tweet', how = 'inner')
Df_final

In [ ]:
#On a 36607 tweets differents dans la base
len(np.unique(Df_final['noeud_tweet']))

In [ ]:
Category = list(np.unique(Df_final[['PostCategory']]))

In [ ]:
#We create a variable for each label

for i in Category:
    Df_final[i] = np.nan
    for j in range(0,len(Df_final)):
        if Df_final.loc[j,'PostCategory'] == i :
            Df_final.loc[j,i] = 1
        else :
            Df_final.loc[j,i] = 0

In [ ]:
#We encode the variable Niveau, Compte_verif and Sensitivite
Df_final['Compte_verif'] = Df_final['Compte_verif'].replace([True,False], [0,1])
Df_final['Sensitivité'] = Df_final['Sensitivité'].replace([True,False], [0,1])
Df_final['Niveau'] = Df_final['Niveau'].replace(['Unknown', 'Low', 'Medium','High','Critical'], [0,1, 2,3,4])


In [ ]:
#WordCloud for the tweets of News
from wordcloud import WordCloud

lis = ''

for tweet in Df_final['Contenu'][Df_final["PostCategory"]== "News"] :
    lis += tweet

wordcloud = WordCloud(width = 800, height = 800, min_font_size = 10).generate(lis)

plt.figure(figsize=(6,6))
plt.title("WordCloud for News")
plt.imshow(wordcloud)
plt.axis("off")

In [ ]:
#We aggregate per tweet

Df_final = Df_final.groupby(['noeud_tweet','Nom perso', 'Compte_verif', 'Nb_followers', 'Nb_favories', 'Nb_emis', 'Nb_public', 'Nom Utilisateur', 'Contenu', 'Niveau', 'Nb_retweet', 'Sensitivité'], as_index=False).sum()
Df_final = Df_final.drop(['noeud_Util','Evenement', 'PostCategory'], axis=1)


In [ ]:
#Study of the proportion of each label
Proportion = list()

for i in Category :
    Proportion.append(Df_final[i].sum()*100/len(Df_final))
Repartition_df = pd.DataFrame({'Category' : Category, 'Freq' : Proportion })
Repartition_df = Repartition_df.sort_values('Freq')
#We create downgrade color for the barplot
color_map = plt.cm.get_cmap('Reds')
colors = color_map(np.linspace(0, 1, len(Proportion)))
#Barplot of proportion of each classes.
plt.barh(Repartition_df['Category'], Repartition_df['Freq'], color = colors)
plt.yticks(fontsize=10)
plt.xlabel('Frequence')

# SPLIT TRAIN TEST DATASET

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
y = np.asarray(Df_final[Df_final.columns[13:38]])
#X = Df_final.drop(Category, axis = 1)
X = Df_final
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=43)
X_train.reset_index(drop=True, inplace = True)
X_test.reset_index(drop=True, inplace = True)

In [ ]:
len(X_test)

In [ ]:
len(X_train)

In [ ]:
#Proportion in the train
for i in Category :
    print('Catgory', i, X_train[i].sum()*100/len(X_train))

In [ ]:
#Proportion in the test
for i in Category :
    print('Catgory', i, X_test[i].sum()*100/len(X_test))

# Reechantillonage : Up et Down Sampling (TRAIN)

In [ ]:
# UpSampling
Propmin = 10
index_CleanUp = X_train[X_train['CleanUp'] == 1]
index_NewSubEvent = X_train[X_train['NewSubEvent'] == 1]
index_Volunteer = X_train[X_train['Volunteer'] == 1]
index_GoodsServices = X_train[X_train['GoodsServices'] == 1]
index_SearchAndRescue = X_train[X_train['SearchAndRescue'] == 1]
index_InformationWanted = X_train[X_train['InformationWanted'] == 1]
index_Donations = X_train[X_train['Donations'] == 1]
index_MovePeople = X_train[X_train['MovePeople'] == 1]

AM1 = index_CleanUp.sample(round(((Propmin - 0.6) / 100 * len(X_train))), random_state=42, replace=True).index
AM2 = index_NewSubEvent.sample(round((Propmin - 2.5) / 100 * len(X_train)), random_state=42, replace=True).index
AM4 = index_Volunteer.sample(round(((Propmin - 0.57) / 100 * len(X_train))), random_state=42, replace=True).index
AM5 = index_GoodsServices.sample(round((Propmin - 0.50) / 100 * len(X_train)), random_state=42, replace=True).index
AM8 = index_SearchAndRescue.sample(round((Propmin - 0.71) / 100 * len(X_train)), random_state=42, replace=True).index
AM9 = index_InformationWanted.sample(round((Propmin - 0.76) / 100 * len(X_train)), random_state=42, replace=True).index
AM11 = index_Donations.sample(round(((Propmin - 2.6) / 100 * len(X_train))), random_state=42, replace=True).index
AM13 = index_MovePeople.sample(round((Propmin - 1) / 100 * len(X_train)), random_state=42, replace=True).index

AMETTRE = list((list(AM1) + list(AM2) + list(AM4) + list(AM5) + list(AM8) + list(AM9) + list(AM11) + list(AM13)))
X_train_resample = X_train.loc[AMETTRE]
X_train_resample = pd.concat([X_train, X_train_resample])

# Resampling pour rebalancer les classes : Downsampling
Propmax = 20
index_Irrelevant = X_train[X_train['Irrelevant'] == 1]
index_ThirdPartyObservation = X_train[X_train['ThirdPartyObservation'] == 1]
index_FirstPartyObservation = X_train[X_train['FirstPartyObservation'] == 1]
index_Hashtags = X_train[X_train['Hashtags'] == 1]
index_Location = X_train[X_train['Location'] == 1]
index_MultimediaShare = X_train[X_train['MultimediaShare'] == 1]
index_Sentiment = X_train[X_train['Sentiment'] == 1]
index_News = X_train[X_train['News'] == 1]

AR2 = index_Sentiment.sample(round((23 - Propmax) / 100 * len(X_train_resample)), random_state=42, replace=True).index
AR4 = index_MultimediaShare.sample(round((26 - Propmax) / 100 * len(X_train_resample)), random_state=42, replace=True).index
AR5 = index_Location.sample(round((22 - Propmax) / 100 * len(X_train_resample)), random_state=42, replace=True).index
AR7 = index_Hashtags.sample(round((31 - Propmax) / 100 * len(X_train_resample)), random_state=42, replace=True).index
AR8 = index_News.sample(round((29 - Propmax) / 100 * len(X_train_resample)), random_state=42, replace=True).index
ASUPP = list(set(list(AR2) + list(AR4) + list(AR2) + list(AR5) + list(AR7) + list(AR8)))
X_train_resample = X_train_resample.drop(ASUPP)

X_train = X_train_resample
X_train.reset_index(drop=True, inplace=True)

In [ ]:
#Study of the proportion of each label
Proportion = list()

for i in Category :
    Proportion.append(X_train[i].sum()*100/len(X_train))
Repartition_df = pd.DataFrame({'Category' : Category, 'Freq' : Proportion })
Repartition_df = Repartition_df.sort_values('Freq')
#We create downgrade color for the barplot
color_map = plt.cm.get_cmap('Reds')
colors = color_map(np.linspace(0, 1, len(Proportion)))
#Barplot of proportion of each classes.
plt.barh(Repartition_df['Category'], Repartition_df['Freq'], color = colors)
plt.yticks(fontsize=10)
plt.xlabel('Frequence for the train set')

# Reechantillonage : Up et Down Sampling (TEST)

In [ ]:
# UpSampling
Propmin = 10
index_CleanUp = X_test[X_test['CleanUp'] == 1]
index_NewSubEvent = X_test[X_test['NewSubEvent'] == 1]
index_Volunteer = X_test[X_test['Volunteer'] == 1]
index_GoodsServices = X_test[X_test['GoodsServices'] == 1]
index_SearchAndRescue = X_test[X_test['SearchAndRescue'] == 1]
index_InformationWanted = X_test[X_test['InformationWanted'] == 1]
index_Donations = X_test[X_test['Donations'] == 1]
index_MovePeople = X_test[X_test['MovePeople'] == 1]

AM1 = index_CleanUp.sample(round(((Propmin - 0.6) / 100 * len(X_test))), random_state=42, replace=True).index
AM2 = index_NewSubEvent.sample(round((Propmin - 2.5) / 100 * len(X_test)), random_state=42, replace=True).index
AM4 = index_Volunteer.sample(round(((Propmin - 0.57) / 100 * len(X_test))), random_state=42, replace=True).index
AM5 = index_GoodsServices.sample(round((Propmin - 0.50) / 100 * len(X_test)), random_state=42, replace=True).index
AM8 = index_SearchAndRescue.sample(round((Propmin - 0.71) / 100 * len(X_test)), random_state=42, replace=True).index
AM9 = index_InformationWanted.sample(round((Propmin - 0.76) / 100 * len(X_test)), random_state=42, replace=True).index
AM11 = index_Donations.sample(round(((Propmin - 2.6) / 100 * len(X_test))), random_state=42, replace=True).index
AM13 = index_MovePeople.sample(round((Propmin - 1) / 100 * len(X_test)), random_state=42, replace=True).index

AMETTRE = list((list(AM1) + list(AM2) + list(AM4) + list(AM5) + list(AM8) + list(AM9) + list(AM11) + list(AM13)))
X_test_resample = X_test.loc[AMETTRE]
X_test_resample = pd.concat([X_test, X_test_resample])

# Resampling pour rebalancer les classes : Downsampling
Propmax = 20
index_Irrelevant = X_test[X_test['Irrelevant'] == 1]
index_ThirdPartyObservation = X_test[X_test['ThirdPartyObservation'] == 1]
index_FirstPartyObservation = X_test[X_test['FirstPartyObservation'] == 1]
index_Hashtags = X_test[X_test['Hashtags'] == 1]
index_Location = X_test[X_test['Location'] == 1]
index_MultimediaShare = X_test[X_test['MultimediaShare'] == 1]
index_Sentiment = X_test[X_test['Sentiment'] == 1]
index_News = X_test[X_test['News'] == 1]

AR2 = index_Sentiment.sample(round((23 - Propmax) / 100 * len(X_test_resample)), random_state=42, replace=True).index
AR4 = index_MultimediaShare.sample(round((26 - Propmax) / 100 * len(X_test_resample)), random_state=42, replace=True).index
AR5 = index_Location.sample(round((22 - Propmax) / 100 * len(X_test_resample)), random_state=42, replace=True).index
AR7 = index_Hashtags.sample(round((31 - Propmax) / 100 * len(X_test_resample)), random_state=42, replace=True).index
AR8 = index_News.sample(round((29 - Propmax) / 100 * len(X_test_resample)), random_state=42, replace=True).index
ASUPP = list(set(list(AR2) + list(AR4) + list(AR2) + list(AR5) + list(AR7) + list(AR8)))
X_test_resample = X_test_resample.drop(ASUPP)

X_test = X_test_resample
X_test.reset_index(drop=True, inplace=True)

In [ ]:
Proportion_resample = list()

for i in Category :
    Proportion_resample.append(X_test[i].sum()*100/len(X_test))

Repartition_df_resample = pd.DataFrame({'Category' : Category, 'Freq' : Proportion_resample })
Repartition_df_resample = Repartition_df_resample.sort_values('Freq')
#We create downgrade color for the barplot
color_map = plt.cm.get_cmap('Reds')
colors = color_map(np.linspace(0, 1, len(Proportion_resample)))
#Barplot of proportion of each classes.
plt.barh(Repartition_df_resample['Category'], Repartition_df_resample['Freq'], color = colors)
plt.yticks(fontsize=10)
plt.xlabel('Frequence for the test set')

In [ ]:
len(X_train)

In [ ]:
len(np.unique(X_train['noeud_tweet']))

In [ ]:
len(X_test)

In [ ]:
len(np.unique(X_test['noeud_tweet']))

In [ ]:
#Reattribute the label to y
y_train = np.asarray(X_train[X_train.columns[13:38]])
y_test = np.asarray(X_test[X_test.columns[13:38]])
X_train = X_train.drop(Category, axis = 1)
X_test = X_test.drop(Category, axis = 1)

# Create new features

In [ ]:

#Count the number of emoticon
def count_emoticons(text):
    pattern = r'[:;=x][-]?[)D(opd]'
    matches = re.findall(pattern, text)
    return len(matches)

#If there is a link
def link(data, col):
    data['link'] = data[col].str.contains('http')
    data['link'] =data['link'].astype(int)


#Count the number of #
def count_hash(text):
    pattern = r'[#]'
    matches = re.findall(pattern, text)
    return len(matches)

#Count the nb of number
def count_nb(text):
    pattern = r'[1-9]'
    matches = re.findall(pattern, text)
    return len(matches)

#Tell if there is a ponctuation ! or ?
def ponctuation_nb(text):
    pattern = r'[!?]'
    matches = re.findall(pattern, text)
    return len(matches)


#Apply the different function created on our different dataset.
X_train['emoticon_count'] = X_train['Contenu'].apply(count_emoticons)
X_train['hashtags_count'] = X_train['Contenu'].apply(count_hash)
X_train['nb_count'] = X_train['Contenu'].apply(count_nb)
X_train['Ponctuation'] = X_train['Contenu'].apply(ponctuation_nb)
X_test['emoticon_count'] = X_test['Contenu'].apply(count_emoticons)
X_test['hashtags_count'] = X_test['Contenu'].apply(count_hash)
X_test['nb_count'] = X_test['Contenu'].apply(count_nb)
X_test['Ponctuation'] = X_test['Contenu'].apply(ponctuation_nb)
link(X_train,'Contenu')
link(X_test,'Contenu')

In [ ]:
#Store the list of stop words
stop_words = set(stopwords.words('english'))

X_train['Clean_text'] = ""
# We drop the link, the stop words
for index, sentence in enumerate(X_train['Contenu']):
    sentence = re.sub(r'http\S+', " ", sentence)
    sentence = re.sub(r'[@#-]\S+', " ", sentence)
    sentence = re.sub(r'RT', " ", sentence)
    sentence = re.sub(r"\'s", " ", sentence)
    sentence = re.sub(r"\'m", " am ", sentence)
    sentence = re.sub(r"\'ve", " have ", sentence)
    sentence = re.sub(r"n't", " not ", sentence)
    sentence = re.sub(r"\'re", " are ", sentence)
    sentence = re.sub(r"\'ll", " will ", sentence)
    sentence = re.sub(r"\'d", " would ", sentence)
    sentence = re.sub(r"can't", "can not ", sentence)
    sentence = re.sub('[^a-zA-Z]', ' ', sentence) #Caracter without letters
    # Tokenization
    words = word_tokenize(sentence)
    words = [word.lower() for word in words]
    # We drop the stop words and the punctuation
    filtered_words = [word for word in words if word.lower() not in stop_words and word.lower() not in string.punctuation]
    filtered_sentence = ' '.join(filtered_words)
    X_train.loc[index,'Clean_text'] = filtered_sentence

X_test['Clean_text'] = ""
# We drop the link, the stop words
for index, sentence in enumerate(X_test['Contenu']):
    sentence = re.sub(r'http\S+', " ", sentence)
    sentence = re.sub(r'[@#-]\S+', " ", sentence)
    sentence = re.sub(r'RT', " ", sentence)
    sentence = re.sub(r"\'s", " ", sentence)
    sentence = re.sub(r"\'m", " am ", sentence)
    sentence = re.sub(r"\'ve", " have ", sentence)
    sentence = re.sub(r"n't", " not ", sentence)
    sentence = re.sub(r"\'re", " are ", sentence)
    sentence = re.sub(r"\'ll", " will ", sentence)
    sentence = re.sub(r"\'d", " would ", sentence)
    sentence = re.sub(r"can't", "can not ", sentence)
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    # Tokenization
    words = word_tokenize(sentence)
    words = [word.lower() for word in words]
    # We drop the stop words and the punctuation
    filtered_words = [word for word in words if word.lower() not in stop_words and word.lower() not in string.punctuation]
    filtered_sentence = ' '.join(filtered_words)
    X_test.loc[index,'Clean_text'] = filtered_sentence


In [ ]:
# X_train['Clean_Nom'] = ""
# for index, sentence in enumerate(X_train['Nom Utilisateur']):
#     sentence = re.sub(r'http\S+', " ", sentence)
#     # Tokenization des mots dans la phrase
#     words = word_tokenize(sentence)
#     # Filtrage des mots qui ne sont pas des stopwords
#     filtered_words = [word for word in words if word.lower() not in stop_words and word.lower() not in string.punctuation]
#     # Reconstitution de la phrase sans les stopwords
#     filtered_sentence = ' '.join(filtered_words)
#     # Ajout de la phrase filtrée à la nouvelle colonne de texte
#     X_train.loc[index,'Clean_Nom'] = filtered_sentence

# X_test['Clean_Nom'] = ""
# for index, sentence in enumerate(X_test['Nom Utilisateur']):
#     sentence = re.sub(r'http\S+', " ", sentence)
#     # Tokenization des mots dans la phrase
#     words = word_tokenize(sentence)
#     # Filtrage des mots qui ne sont pas des stopwords
#     filtered_words = [word for word in words if word.lower() not in stop_words and word.lower() not in string.punctuation]
#     # Reconstitution de la phrase sans les stopwords
#     filtered_sentence = ' '.join(filtered_words)
#     # Ajout de la phrase filtrée à la nouvelle colonne de texte
#     X_test.loc[index,'Clean_Nom'] = filtered_sentence

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()
#Negativ polarity of the tweet
def neg_sentiment(text):
    sentiment_score = sia.polarity_scores(text)
    return sentiment_score['neg']
#Positive polarity of the tweet
def pos_sentiment(text):
    sentiment_score = sia.polarity_scores(text)
    return sentiment_score['pos']
#Neutral polarity of the tweet
def neutral_sentiment(text):
    sentiment_score = sia.polarity_scores(text)
    return sentiment_score['neu']
X_train['Positive polarity'] = X_train['Contenu'].apply(pos_sentiment)
X_train['Neutral polarity'] = X_train['Contenu'].apply(neutral_sentiment)
X_train['Negative_polarity'] = X_train['Contenu'].apply(neg_sentiment)
X_test['Positive polarity'] = X_test['Contenu'].apply(pos_sentiment)
X_test['Neutral polarity'] = X_test['Contenu'].apply(neutral_sentiment)
X_test['Negative_polarity'] = X_test['Contenu'].apply(neg_sentiment)


In [ ]:
#We compute the length of clean text
X_train['longueur'] = X_train['Clean_text'].apply(len)
X_test['longueur'] = X_test['Clean_text'].apply(len)
X_train

# OPTION 1 : SENTENCE EMBEDDING

This option for make word embedding perform well

In [ ]:

# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer("all-MiniLM-L6-v2")

# # for index, sentence in enumerate(Df_final['Clean_text']):
# #     Df_final.loc[index,'mean_embedding'] = list(model.encode(sentence))

# X_train['mean'] = X_train['Clean_text'].apply(lambda x : list(model.encode(x)))

# X_test['mean'] = X_test['Clean_text'].apply(lambda x : list(model.encode(x)))

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

X_train['mean'] = X_train['Clean_text'].apply(lambda x : list(sbert_model.encode(x)))

X_test['mean'] = X_test['Clean_text'].apply(lambda x : list(sbert_model.encode(x)))

In [ ]:
def extract_elements(row):
    return pd.Series(row['mean'])

# Appliquer la fonction de transformation sur la colonne et créer les nouvelles colonnes
new_columns = X_train.apply(extract_elements, axis=1)

# Ajouter les nouvelles colonnes au DataFrame original
X_train = pd.concat([X_train, new_columns], axis=1)

# Renommer les nouvelles colonnes si nécessaire
X_train.rename(columns={i: f'Embedding_tweet_sentence{i+1}' for i in range(len(new_columns.columns))}, inplace=True)

new_columns = X_test.apply(extract_elements, axis=1)

# Ajouter les nouvelles colonnes au DataFrame original
X_test = pd.concat([X_test, new_columns], axis=1)

# Renommer les nouvelles colonnes si nécessaire
X_test.rename(columns={i: f'Embedding_tweet_sentence{i+1}' for i in range(len(new_columns.columns))}, inplace=True)

In [ ]:

X_train =X_trainbis
X_test = X_testbis

# WORD EMBEDDING

In [ ]:
# sentences = [word_tokenize(sentence) for sentence in Df_final['Clean_text']]
# model = Word2Vec(sentences=sentences, vector_size=50, window=5, min_count=1, workers=4)
# Df_final['embeddings_Contenu'] = Df_final['Clean_text'].apply(lambda x: [model.wv[word] for word in x if word in model.wv])
# vocabulary = list(model.wv.index_to_key)

In [ ]:
# sentences_nom = [word_tokenize(sentence) for sentence in X_train['Nom Utilisateur']]
# model_nom = Word2Vec(sentences=sentences_nom, vector_size=50, window=5, min_count=1, workers=4)
# X_train['embeddings_Contenu_Nom'] = X_train['Clean_Nom'].apply(lambda x: [model_nom.wv[word] for word in x if word in model_nom.wv])
# list(model_nom.wv.index_to_key)

# sentences_nom = [word_tokenize(sentence) for sentence in X_test['Nom Utilisateur']]
# model_nom = Word2Vec(sentences=sentences_nom, vector_size=50, window=5, min_count=1, workers=4)
# X_test['embeddings_Contenu_Nom'] = X_test['Clean_Nom'].apply(lambda x: [model_nom.wv[word] for word in x if word in model_nom.wv])
# list(model_nom.wv.index_to_key)

In [ ]:
# def calculate_mean_embedding(embeddings):
#     if embeddings:
#         return np.sum(embeddings, axis=0)
#     else:
#         # Si aucun mot n'a d'embedding, retourner un vecteur de zéros
#         return np.zeros(50)  # Assurez-vous que la taille du vecteur correspond à la dimension de vos embeddings

# # Ajouter une nouvelle colonne avec les moyennes des embeddings
# #Df_final['mean_embedding'] = Df_final['embeddings_Contenu'].apply(calculate_mean_embedding)
# X_train['mean_embedding_Nom'] = X_train['embeddings_Contenu_Nom'].apply(calculate_mean_embedding)
# X_test['mean_embedding_Nom'] = X_test['embeddings_Contenu_Nom'].apply(calculate_mean_embedding)

In [ ]:
# def extract_elements(row):
#     return pd.Series(row['mean_embedding'])

# # Appliquer la fonction de transformation sur la colonne et créer les nouvelles colonnes
# new_columns = Df_final.apply(extract_elements, axis=1)

# # Ajouter les nouvelles colonnes au DataFrame original
# Df_final = pd.concat([Df_final, new_columns], axis=1)

# # Renommer les nouvelles colonnes si nécessaire
# Df_final.rename(columns={i: f'Embedding_tweet{i+1}' for i in range(len(new_columns.columns))}, inplace=True)

In [ ]:
# def extract_elements_Nom(row):
#     return pd.Series(row['mean_embedding_Nom'])

# # Appliquer la fonction de transformation sur la colonne et créer les nouvelles colonnes
# new_columns = X_train.apply(extract_elements_Nom, axis=1)

# # Ajouter les nouvelles colonnes au DataFrame original
# X_train = pd.concat([X_train, new_columns], axis=1)

# # Renommer les nouvelles colonnes si nécessaire
# X_train.rename(columns={i: f'Embedding_Nom{i+1}' for i in range(len(new_columns.columns))}, inplace=True)

# # Appliquer la fonction de transformation sur la colonne et créer les nouvelles colonnes
# new_columns = X_test.apply(extract_elements_Nom, axis=1)

# # Ajouter les nouvelles colonnes au DataFrame original
# X_test = pd.concat([X_test, new_columns], axis=1)

# # Renommer les nouvelles colonnes si nécessaire
# X_test.rename(columns={i: f'Embedding_Nom{i+1}' for i in range(len(new_columns.columns))}, inplace=True)

# Node embedding

We have to drop edges between tweet and postcategory wether the model will be biais.

In [ ]:
#We remove nodes which are Category
nodes_to_remove = []


for node, attributes in graph.nodes(data=True):
    if attributes.get('labels') == ':PostCategory':
        nodes_to_remove.append(node)

for node in nodes_to_remove:
    graph.remove_node(node)


In [ ]:
p = 1 # Paramètre de probabilité de retour arrière
q = 0.7  # Paramètre de contrôle de l'exploration
dimensions = 100  # Taille de l'embedding
num_walks = 80  # Nombre de promenades aléatoires par nœud
walk_length = 90  # Longueur des promenades aléatoires
tweet_embeddings={}

node2vec = Node2Vec(graph, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, p=p, q=q)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

for node, attributes in graph.nodes(data=True):
    if attributes.get('labels') == ':Tweet':
        tweet_embeddings[node] = model.wv[node]


In [ ]:
embedding_dim = 100
for i in range(embedding_dim):
    X_train[f"embedding_{i+1}"] = None
    X_test[f"embedding_{i+1}"] = None

In [ ]:
for i, row in X_train.iterrows():
    node = row['noeud_tweet']
    if node in tweet_embeddings:
        embedding = tweet_embeddings[node]
        for j, val in enumerate(embedding):
            X_train.at[i, f"embedding_{j+1}"] = val

for i, row in X_test.iterrows():
    node = row['noeud_tweet']
    if node in tweet_embeddings:
        embedding = tweet_embeddings[node]
        for j, val in enumerate(embedding):
            X_test.at[i, f"embedding_{j+1}"] = val

In [ ]:
X_train

In [ ]:
X_train = X_train.drop(['noeud_tweet', 'Clean_text', 'Contenu', 'Nom perso', 'Nom Utilisateur',  'mean'], axis=1)
X_test = X_test.drop(['noeud_tweet', 'Clean_text', 'Contenu', 'Nom perso', 'Nom Utilisateur',  'mean'], axis=1)


#Scaler
scaler = MinMaxScaler()

# Normalize
X_traindd = scaler.fit_transform(X_train)
X_testdd = scaler.fit_transform(X_test)

# Remettre le résultat dans un dataframe avec les mêmes colonnes
X_train = pd.DataFrame(X_traindd, columns=X_train.columns)
X_test = pd.DataFrame(X_testdd, columns=X_test.columns)

In [ ]:
X_train

# Multi Label Classification : Logistic

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


clf = MultiOutputClassifier(LogisticRegression()).fit(X_train[X_train.columns[:985]], y_train)


In [ ]:
y_pred = clf.predict(X_test[X_test.columns[:985]])
y_pred
f1 = f1_score(y_test, y_pred,average='micro')
f1

In [ ]:
df_cat

In [ ]:
# Calculez le F1-score par catégorie
f1_scores = f1_score(y_test, y_pred, average=None)

score =list()
# Affichez les F1-scores par catégorie
for i, label in enumerate(Category):
    score.append(f1_scores[i])
    print(f"F1-score for category {label}: {f1_scores[i]}")

df_cat = pd.DataFrame({'Cat' : Category, 'F1' : score}, index = range(1,26))
df_cat =df_cat.sort_values('F1')
plt.barh(df_cat['Cat'], df_cat['F1'], color =colors)
plt.xlabel('F1 Score Logistic')

# MULTI LABEL : RANDOM FOREST CLASSIFIER

In [ ]:
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
model = MultiOutputClassifier(RandomForestClassifier(n_estimators = 100))
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the F1-score with micro average
f1 = f1_score(y_test, y_pred,average='micro')

print("The F1-score is equal to:", f1)

# OPTIMISATION BAESIENNE

In [ ]:
#1ere approche HyperTunning avec Grid Search
from sklearn.model_selection import GridSearchCV
rf_params = { "n_estimators" :[300,400,600],
            "max_depth" : [8,11,15],
            "min_samples_split" : [2,5,8]}
rf_model = GridSearchCV(model, rf_params, cv =3, n_jobs = -1, verbose = 2).fit(X_train, y_train)
rf_model.best_params_

In [ ]:
#Code :https://www.kaggle.com/code/adityarawat0701/xgboost-multilabel-classification-bayesian-opt
from bayes_opt import BayesianOptimization
def xgb_cv(max_depth, learning_rate, subsample, colsample_bytree,min_child_weight):
    params = {'objective': 'binary:logistic',
              'max_depth': int(max_depth),
              'learning_rate': learning_rate,
              'subsample': subsample,
              'min_child_weight': min_child_weight,
              'colsample_bytree': colsample_bytree}
    dtrain = xgb.DMatrix(X_train, label=y_train)
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, early_stopping_rounds=10, nfold=5, metrics='error')
    return -cv_result['test-error-mean'].iloc[-1]

pbounds = {'max_depth': (3, 9),
           'learning_rate': (0.01, 0.5),
           'subsample': (0.1, 1),
           'colsample_bytree': (0.1, 1),
           'min_child_weight': (1, 12)
          }

print('Performing hyperparameter tuning using Bayesian optimization...')
optimizer = BayesianOptimization(f=xgb_cv, pbounds=pbounds, random_state=1)
optimizer.maximize(init_points=5, n_iter=20)

# XG BOOST

In [ ]:
import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
xgbb = xgb.XGBClassifier()
xgbb.fit(X_train,y_train)

y_pred = xgbb.predict(X_test)

#Calculate the F1-score
f1 = f1_score(y_test, y_pred,average='micro')
f1

In [ ]:
# Calculez le F1-score par catégorie
f1_scores = f1_score(y_test, y_pred, average=None)

score =list()
# Affichez les F1-scores par catégorie
for i, label in enumerate(Category):
    score.append(f1_scores[i])
    print(f"F1-score for category {label}: {f1_scores[i]}")

df_cat = pd.DataFrame({'Cat' : Category, 'F1' : score}, index = range(1,26))
df_cat =df_cat.sort_values('F1')
plt.barh(df_cat['Cat'], df_cat['F1'], color =colors)
plt.xlabel('F1 Score XGBoost')

In [ ]:
from sklearn.model_selection import GridSearchCV
rf_params = {"learning_rate": [0.05, 0.1,0.3],
            "max_depth" : [7,9,5]}
rf_model = GridSearchCV(xgbb, rf_params, cv =3, n_jobs = -1, verbose = 2).fit(X_train, y_train)
rf_model.best_params_

In [ ]:
xgbb = xgb.XGBClassifier(n_estimators =300)
xgbb.fit(X_train,y_train)

y_pred = xgbb.predict(X_test)

#Calculate the F1-score
f1 = f1_score(y_test, y_pred,average='micro')
f1

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot
plt.figsize(15,15)
plot_importance(xgbb)
plt.rcParams['ytick.labelsize'] = 0.1
pyplot.show()

In [ ]:
last_20_indices

In [ ]:
importance = xgbb.feature_importances_

# Get the indices of the last 20 variables
last_20_indices = importance.argsort()[-100:]

# Get the names of the last 20 variables
last_20_features = [f'Feature {i}' for i in last_20_indices]

# Plot the importance of the last 20 variables
plt.figure(figsize=(10, 6))
plt.barh(range(len(last_20_indices)), importance[last_20_indices], align='center')
plt.yticks(range(len(last_20_indices)), last_20_features, fontsize=12)
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.title('Importance of Last 20 Variables')
plt.show()

# DEEP LEARNING : RESEAU DE NEURONNES

In [ ]:

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.metrics import F1Score
import keras


model =Sequential()
model.add(Dense(1000, activation='relu', input_dim = X_train.shape[1] ))
model.add(Dense(400, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation ='relu'))
model.add(Dense(25,activation ='sigmoid')
)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])


# Entraîner le modèle
#model.fit(X_train, y_train, epochs=100, batch_size = 128)
history = model.fit(X_train, y_train,validation_split=0.34,shuffle=True,epochs=100,batch_size=150,verbose=0)
metrics = model.evaluate(X_test, y_test)




In [ ]:
# Calculez le F1-score par catégorie
y_pred = model.predict(X_test)

# Binariser les prédictions et les étiquettes réelles (nécessaire pour le calcul du F1-score)
y_pred_binary = np.round(y_pred)
y_test_binary = y_test

f1_scores = f1_score(y_test, y_pred_binary, average=None)

score =list()
# Affichez les F1-scores par catégorie
for i, label in enumerate(Category):
    score.append(f1_scores[i])
    print(f"F1-score for category {label}: {f1_scores[i]}")

df_cat = pd.DataFrame({'Cat' : Category, 'F1' : score}, index = range(1,26))
df_cat =df_cat.sort_values('F1')
plt.barh(df_cat['Cat'], df_cat['F1'], color =colors)
plt.xlabel('F1 Score Neural Network')

In [ ]:
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('binary_accuracy')
plt.xlabel('epoch')
plt.yticks(fontsize = 12)
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


In [ ]:
y_pred = model.predict(X_test)

# Binariser les prédictions et les étiquettes réelles (nécessaire pour le calcul du F1-score)
y_pred_binary = np.round(y_pred)
y_test_binary = y_test

# Calculer le F1-score pour chaque classe (étiquette) et calculer la moyenne
f1_scores = f1_score(y_test_binary, y_pred_binary, average='micro')

print("Le score F1 est égal à:", f1_scores)

In [ ]:
model.summary()

In [ ]:
f1_scores = f1_score(y_test_binary, y_pred_binary, average=None)

# Affichez les F1-scores par catégorie
for i, label in enumerate(Category):
    print(f"F1-score for category {label}: {f1_scores[i]}")